# An example of creating coefficients using pyEXP
This example assumes that you have run the `EXP/examples/Better` simulation.  But this notebook could be adapted for any simulation you like.

We begin by importing `pyEXP` and friends and setting the working directory.

In [1]:
import os
import yaml
import pyEXP

# I have the Better run here; obviously another use will want to
# change this a directory containing some snapshots of their own
#
os.chdir('/home/weinberg/Nbody/Better')

## Create the basis
We'll only do the halo coefficients in this simple example.  The cylindrical coefficients would procede similarly.  See the `sample_basis` notebook for an example of creating the cylindrical basis.

In [2]:
# Get the basis config
#
yaml_config = ""
with open('basis.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    yaml_config = yaml.dump(config)

# Alternatively, you could construct this on the fly, e.g.
bconfig = """
---
id: sphereSL
parameters :
  numr: 2000
  rmin: 0.0001
  rmax: 1.95
  Lmax: 4
  nmax: 10
  rs: 0.0667
  modelname: SLGridSph.model
...
"""
print('-'*60)
print('Read from file')
print('-'*60)
print(yaml_config)
print('-'*60)
print('Constructed from string')
print('-'*60)
print(bconfig)
print('-'*60)

# Construct the basis instance
#
basis   = pyEXP.basis.Basis.factory(yaml_config)

------------------------------------------------------------
Read from file
------------------------------------------------------------
id: sphereSL
parameters:
  Lmax: 4
  modelname: SLGridSph.model
  nmax: 10
  numr: 2000
  rmax: 1.95
  rmin: 0.0001
  rs: 0.0667

------------------------------------------------------------
Constructed from string
------------------------------------------------------------

---
id: sphereSL
parameters :
  numr: 2000
  rmin: 0.0001
  rmax: 1.95
  Lmax: 4
  nmax: 10
  rs: 0.0667
  modelname: SLGridSph.model
...

------------------------------------------------------------
---- SLGridSph::read_cached_table: trying to read cached table . . .
---- SLGridSph::read_cached_table: Success!!


## Creating a particle reader
Now that we have a basis, we can use it to create coefficients from the particle snapshots.  `pyEXP` uses a `ParticleReader` object for that.

The first step is to hand off the files that comprise a snapshot for every time slice.  The `ParticleReader` provides a helper function for that.   There are two helper functions: `parseFileList` and `parseStringList`.  The first reads a list from a file and the second takes a list.  Otherwise they are the same.  The file names in the list are assumed to end with a snapshot index and an optional part index.  For example, if you have single files per snapshot, the list might look like: `myrun.00000`, `myrun.00001`, etc.  If you have multiple files per snapshot, they will look something like `myrun.00000_0001`, `myrun.00000_0002`, `myrun.00001_0000`, `myrun.00001_0001`, etc.

Here is the call for a file:

In [3]:
# Construct batches of files the particle reader.  One could use the
# parseStringList to create batches from a vector/list of files.  NB:
# a std::vector in C++ becomes a Python.list and vice versa
#
batches = pyEXP.read.ParticleReader.parseFileList('file.list', '')

This is my test callback function.  It will print the first 10 particles just to convince us that it is working, 

In [4]:
def selection(mass, pos, vel, index):
    if index<=10:
        print('index, mass, x, y, z =', index, mass, pos[0], pos[1], pos[2])
    return True

basis.setSelector(selection)

Here is a center of mass functor...

In [7]:
totalMass = 0.0
com = [0.0, 0.0, 0.0]
def makeCOM(mass, pos, vel, index):
    global totalMass, com
    totalMass += mass
    for i in range(3): com[i] += mass * pos[i]

We now iterate the `batches` created by the file parser to create the coefficients.   For each batch we create a new reader and pass the reader to the basis instance.  The `basis.createFromReader` member creates and returns the coefficients.  The coefficients are added to a coefficient container called `coefs`.  Note: on the first call `coefs=None` so a new container is created on the first time through.

In [ ]:
# This will contain the coefficient container, need to start will a
# null instance to trigger construction
#
coefs   = None

for group in batches:

    print("file group is", group)

    # Make the reader for the desired type.  One could probably try to
    # do this by inspection but that's another project.
    #
    reader = pyEXP.read.ParticleReader.createReader('PSPout', group, 0, False);

    # Print the type list
    #
    print('The component names are:', reader.GetTypes())

    compname = 'dark halo'
    reader.SelectType(compname)
    print('Selected', compname)

    print('Call createFromReader at Time', reader.CurrentTime(), 'for', reader.CurrentNumber(), 'particles')

    coef = basis.createFromReader(reader)
    print("Created coef")
    
    print("Testing the density center creation")
   
    center = pyEXP.util.getDensityCenter(reader, stride=1, Nsort=1000)
    print("Density center is", center)
    
    print("Testing the particle iterator by computing COM")
    totalMass = 0.0
    com = [0.0, 0.0, 0.0]
    pyEXP.util.particleIterator(reader, makeCOM)
    print("Center of mass is", com[0]/totalMass, com[1]/totalMass, com[2]/totalMass)

    # We need this stupid idiom here because None is not mapping to a
    # null pointer.  There is probably a way to do this.  Suggestions
    # anyone?
    #                          This is optional---+
    #                                             |
    if coefs is None:           #                 v
        coefs = pyEXP.coefs.Coefs.makecoefs(coef, compname)
    else:
        coefs.add(coef)

    print('Added coef')
    print('-'*60)

print('\nCompleted the file group list\n')

print('The coefficient time list is', coefs.Times())

file group is ['OUT.run0.00000']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.0 for 100000 particles
index, mass, x, y, z = 1 9.651515028963331e-06 0.941630482673645 0.6668369770050049 0.7036028504371643
index, mass, x, y, z = 2 6.243539019124e-07 0.002228023484349251 -0.0017561642453074455 8.763065852690488e-05
index, mass, x, y, z = 3 1.4266431207943242e-05 0.6066200733184814 0.40197044610977173 -0.8750137090682983
index, mass, x, y, z = 4 1.91562830877956e-05 0.18146848678588867 0.019536208361387253 -0.04939988628029823
index, mass, x, y, z = 5 1.0565162483544555e-05 0.9574946761131287 -0.2784556746482849 0.46641644835472107
index, mass, x, y, z = 6 1.897938818729017e-05 -0.10535112023353577 -0.026948699727654457 -0.030037209391593933
index, mass, x, y, z = 7 1.0126675988431089e-05 0.6734440922737122 0.8086175322532654 -0.5085681080818176
index, mass, x, y, z = 8 1.4755632037122268e-05 0.7752076387405396 -1.1270403861999512 0

Created coef
Testing the density center creation
Center is [0.0006564704425051625, 0.0008650703950327944, 0.00011291313616264223]
Testing the particle iterator
Center of mass is -0.0011557407137849163 0.0007887225792629137 0.0013693390839511943
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00006']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.030000000000000023 for 100000 particles
index, mass, x, y, z = 1 9.651515028963331e-06 0.91961270570755 0.6617166996002197 0.6971267461776733
index, mass, x, y, z = 2 6.243539019124e-07 -0.0021510962396860123 -0.005082123447209597 -0.0010174773633480072
index, mass, x, y, z = 3 1.4266431207943242e-05 0.6215982437133789 0.39078521728515625 -0.8613817691802979
index, mass, x, y, z = 4 1.91562830877956e-05 0.2202572226524353 0.014667280949652195 -0.045208048075437546
index, mass, x, y, z = 5 1.0565162483544555e-05 0.9832425713539124 -0.27846673

Created coef
Testing the density center creation
Center is [-0.0008154851560310723, 0.0003480472639414452, 3.548872797666656e-05]
Testing the particle iterator
Center of mass is -0.0011464513282355548 0.0007323071889134345 0.0013794883144118049
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00012']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.060000000000000046 for 100000 particles
index, mass, x, y, z = 1 9.651515028963331e-06 0.8971823453903198 0.6563032865524292 0.690340518951416
index, mass, x, y, z = 2 6.243539019124e-07 -0.0022421381436288357 0.004748258739709854 4.104750041733496e-05
index, mass, x, y, z = 3 1.4266431207943242e-05 0.6361618638038635 0.379339337348938 -0.8471730947494507
index, mass, x, y, z = 4 1.91562830877956e-05 0.2519020140171051 0.009206975810229778 -0.03944063186645508
index, mass, x, y, z = 5 1.0565162483544555e-05 1.0083101987838745 -0.278287768363

## Using a FieldGenerator
Now that we have our new coefficients, we can use the `FieldGenerator` to view the BFE representation of the underlying fields.  Here is an example:

In [ ]:
# Now try some slices for rendering
#

times = coefs.Times()[0:3]
pmin  = [-1.0, -1.0, 0.0]
pmax  = [ 1.0,  1.0, 0.0]
grid  = [  40,   40,   0]

fields = pyEXP.field.FieldGenerator(times, pmin, pmax, grid)

surfaces = fields.slices(basis, coefs)

print("We now have the following [time field] pairs")
for v in surfaces:
    print('-'*40)
    for u in surfaces[v]:
        print("{:8.4f}  {}".format(v, u))

print("\nHere is the first one:")
for v in surfaces:
    for u in surfaces[v]:
        print('-'*40)
        print('----', u)
        print('-'*40)
        print(surfaces[v][u])
    break

These could be make into images and so forth.   We'll do this in another example notebook.

## Saving the coefficients

At this point, it makes sense to save the coefficients that you have just created.  This is sone with the following call:

In [ ]:
coefs.WriteH5Coefs('test_better')

We now have a EXP HDF5 coefficient file called `test_better.h5`.    As an example, let's try reading the newly created file into another coefficient container, `coefs2`.  The container has a member function called `CompareStanzas` which will check on the contents.  Let's do it.

In [ ]:
# Now try reading it in
#
coefs2 = pyEXP.coefs.Coefs.factory('test_better.h5')
print("Type is", coefs2.getGeometry())

# Now compare with the original
#
coefs2.CompareStanzas(coefs)

This member function will print differences.  No differenced should be printed, of course.